# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [5]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-64afa1085b-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Rome explorative search") 

Consider the following exploratory information need:

> The goal of this workflow is to study the Ancient Rome and some of the main events connected to that historical period. For instance, the emperors, killings, wars and so on.

## Useful URIs for the current workflow
The following data are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1048`| Julius Caesar       | node |





Also consider that

```
wd:Q1048 ?p ?obj .
```

is the BGP to retrieve all **properties of Julius Caesar**.

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Ancient Rome

2. Find all the Roman Dictators (return the IRI and the name of the dictator)

3. Identify the BGP for Roman Empire and Roman Republic

4. Return the IRI and the name of all Roman emperors killed by homicide (see "manner of death") and, for each killed emperor, return also the IRI and name of the killer, if present in Wikidata.

5. Return the Roman emperors who killed another Roman emperor (the result set must a list of 4-tuples with killer IRI and label, and killed IRI and label).

6. Find the BGP for war

7. Find the wars (note that "war" and "battle" are not the same thing) participated by the Roman Empire (return IRI and name of the war).

8. Consider all the wars participated by the Roman Empire. For each war return the number of related battles and the opponent (a war is composed of one or more battles) (the result set must be a list of 5-tuples war IRI, label, opponent IRI and label, and #battles).


## Task 1
Identify the BGP for Ancient Rome

In [34]:
# query example
queryString = """
SELECT DISTINCT ?p ?name  ?o ?oname
WHERE {
   # bind something
   wd:Q1048 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oname
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('name', 'position held'), ('o', 'http://www.wikidata.org/entity/Q5413489'), ('oname', 'curule aedile')]
[('p', 'http://www.wikidata.org/prop/direct/P3701'), ('name', 'incarnation of'), ('o', 'http://www.wikidata.org/entity/Q58373'), ('oname', 'Amun')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work'), ('o', 'http://www.wikidata.org/entity/Q544948'), ('oname', 'Commentarii de Bello Civili')]
[('p', 'http://www.wikidata.org/prop/direct/P3716'), ('name', 'social classification'), ('o', 'http://www.wikidata.org/entity/Q16744001'), ('oname', 'noble')]
[('p', 'http://www.wikidata.org/prop/direct/P2358'), ('name', 'Roman praenomen'), ('o', 'http://www.wikidata.org/entity/Q5517401'), ('oname', 'Gaius')]
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('name', 'position held'), ('o', 'http://www.wikidata.org/entity/Q172907'), ('oname', 'praetor')]
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('name',

In [7]:
# Q1747689 --> 'Ancient Rome'
queryString = """
SELECT DISTINCT ?p ?name ?oname
WHERE {
   # bind something
   wd:Q1747689 ?p ?o .
   # get the label
   ?p sc:name ?name.
   ?o sc:name ?oname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P832'), ('name', 'public holiday'), ('oname', 'Saturnalia')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('name', 'history of topic'), ('oname', 'history of ancient Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P1792'), ('name', 'category of associated people'), ('oname', 'Category:Ancient Romans')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('oname', 'Category:Ancient Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent'), ('oname', 'Africa')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('oname', 'Roman Empire')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('oname', 'historical country')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source'), ('oname', 'Granat Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent'),

Final query for this task

In [10]:
# write your final query
# Q1747689 --> 'Ancient Rome'
queryString = """
SELECT DISTINCT ?p ?o ?pname ?oname
WHERE {
   # bind something
   wd:Q1747689 ?p ?o .
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P832'), ('o', 'http://www.wikidata.org/entity/Q207213'), ('pname', 'public holiday'), ('oname', 'Saturnalia')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('o', 'http://www.wikidata.org/entity/Q830852'), ('pname', 'history of topic'), ('oname', 'history of ancient Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P1792'), ('o', 'http://www.wikidata.org/entity/Q5652564'), ('pname', 'category of associated people'), ('oname', 'Category:Ancient Romans')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('o', 'http://www.wikidata.org/entity/Q7098243'), ('pname', "topic's main category"), ('oname', 'Category:Ancient Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('o', 'http://www.wikidata.org/entity/Q15'), ('pname', 'continent'), ('oname', 'Africa')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q2277'), ('pname', 'has part'), ('oname', 'Roman Empire')]
[('p', 'http://ww

## Task 2
Find all the Roman Dictators (return the IRI and the name of the dictator)

In [8]:
# write your queries
# Q1747689 --> 'Ancient Rome'
queryString = """
SELECT DISTINCT ?pname ?p ?o ?oname
WHERE {
   # bind something
   wd:Q1747689 ?p ?o .
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'public holiday'), ('p', 'http://www.wikidata.org/prop/direct/P832'), ('o', 'http://www.wikidata.org/entity/Q207213'), ('oname', 'Saturnalia')]
[('pname', 'history of topic'), ('p', 'http://www.wikidata.org/prop/direct/P2184'), ('o', 'http://www.wikidata.org/entity/Q830852'), ('oname', 'history of ancient Rome')]
[('pname', 'category of associated people'), ('p', 'http://www.wikidata.org/prop/direct/P1792'), ('o', 'http://www.wikidata.org/entity/Q5652564'), ('oname', 'Category:Ancient Romans')]
[('pname', "topic's main category"), ('p', 'http://www.wikidata.org/prop/direct/P910'), ('o', 'http://www.wikidata.org/entity/Q7098243'), ('oname', 'Category:Ancient Rome')]
[('pname', 'continent'), ('p', 'http://www.wikidata.org/prop/direct/P30'), ('o', 'http://www.wikidata.org/entity/Q15'), ('oname', 'Africa')]
[('pname', 'has part'), ('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q2277'), ('oname', 'Roman Empire')]
[('pname', 'insta

In [25]:
# Q2277 --> 'Roman Empire'
queryString = """
SELECT DISTINCT ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q2277 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'contains administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q202311'), ('oname', 'Egypt')]
[('pname', 'currency'), ('o', 'http://www.wikidata.org/entity/Q952064'), ('oname', 'Roman currency')]
[('pname', 'currency'), ('o', 'http://www.wikidata.org/entity/Q208041'), ('oname', 'sestertius')]
[('pname', 'coat of arms'), ('o', 'http://www.wikidata.org/entity/Q163323'), ('oname', 'Roman legion')]
[('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q1456601'), ('oname', 'Category:Roman Empire')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11514315'), ('oname', 'historical period')]
[('pname', 'continent'), ('o', 'http://www.wikidata.org/entity/Q15'), ('oname', 'Africa')]
[('pname', 'has works in the collection'), ('o', 'http://www.wikidata.org/entity/Q160236'), ('oname', 'Metropolitan Museum of Art')]
[('pname', 'basic form of government'), ('o', 'http://www.wikidata.org/entity/Q173424'), ('oname', 'auto

In [4]:
# Q830852 --> 'history of ancient Rome'
# Q201038 --> 'Roman Kingdom'
# Q18287233 --> 'Rome'
# Q2277 --> 'Roman Empire'

queryString = """
SELECT DISTINCT ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q41493 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q66890318'), ('oname', 'Helsinki term bank for the arts and sciences')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q11768'), ('oname', 'Ancient Egypt')]
[('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2041543'), ('oname', 'Ottův slovník naučný')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q269678'), ('oname', 'ancient Near East')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q309'), ('oname', 'history')]
[('pname', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q309'), ('oname', 'history')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1047113'), ('oname', 'specialty')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q486761'), ('oname', 'classical antiquity')]
[('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q486761'), ('oname', 'classical antiquity')]
[('pname', 'fo

In [14]:
# Q830852 --> 'history of ancient Rome' 
# Q201038 --> 'Roman Kingdom' 
# Q18287233 --> 'Rome' 
# Q294846 --> 'Decimus Junius Brutus Albinus' 
# Q5413489 --> 'curule aedile' 
# Q2277 --> 'Roman Empire'
# Q1048 --> 'Julius Caesar' 
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   #Get the data of julius Caesar from the given data
   wd:Q1048 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
   Filter (REGEX (?oname ,"Roman*"))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q99899510'), ('oname', 'Ancient Roman historian')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q98103687'), ('oname', 'Ancient Roman military personnel')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q3409374'), ('oname', 'Ancient Roman priest')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q97667506'), ('oname', 'Ancient Roman politician')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q657438'), ('oname', 'Roman triumph')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pname', 'time period'), ('o', 'http://www.wikidata.org/entity/Q2815472'), ('oname', 'Late Roman Republic')]
[

In [17]:
# Q236885 --> 'Roman dictator' 
# P39 --> 'position held' 

queryString = """
SELECT DISTINCT ?item ?oname
WHERE {
   # bind something
   ?item wdt:P39 wd:Q236885 .
   # get the label
   #?p sc:name ?pname.
   ?item sc:name ?oname
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('item', 'http://www.wikidata.org/entity/Q155312'), ('oname', 'Gaius Claudius Centho')]
[('item', 'http://www.wikidata.org/entity/Q297783'), ('oname', 'Appius Claudius Caecus')]
[('item', 'http://www.wikidata.org/entity/Q703448'), ('oname', 'Marcus Livius Salinator')]
[('item', 'http://www.wikidata.org/entity/Q529867'), ('oname', 'Titus Lartius')]
[('item', 'http://www.wikidata.org/entity/Q15034789'), ('oname', 'Manius Valerius Maximus')]
[('item', 'http://www.wikidata.org/entity/Q359810'), ('oname', 'Lucius Caecilius Metellus')]
[('item', 'http://www.wikidata.org/entity/Q929498'), ('oname', 'Quintus Caecilius Metellus')]
[('item', 'http://www.wikidata.org/entity/Q519186'), ('oname', 'Quintus Hortensius')]
[('item', 'http://www.wikidata.org/entity/Q637596'), ('oname', 'Quintus Servilius Priscus Structus Fidenas')]
[('item', 'http://www.wikidata.org/entity/Q1223522'), ('oname', 'Caius Poetelius Libo Visolus')]
[('item', 'http://www.wikidata.org/entity/Q682087'), ('oname', 'Gaiu

Final query for this task

In [18]:
# write your final query
queryString = """
SELECT DISTINCT ?item ?oname
WHERE {
   # bind something
   ?item wdt:P39 wd:Q236885 .
   # get the label
   #?p sc:name ?pname.
   ?item sc:name ?oname
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('item', 'http://www.wikidata.org/entity/Q155312'), ('oname', 'Gaius Claudius Centho')]
[('item', 'http://www.wikidata.org/entity/Q297783'), ('oname', 'Appius Claudius Caecus')]
[('item', 'http://www.wikidata.org/entity/Q703448'), ('oname', 'Marcus Livius Salinator')]
[('item', 'http://www.wikidata.org/entity/Q529867'), ('oname', 'Titus Lartius')]
[('item', 'http://www.wikidata.org/entity/Q15034789'), ('oname', 'Manius Valerius Maximus')]
[('item', 'http://www.wikidata.org/entity/Q359810'), ('oname', 'Lucius Caecilius Metellus')]
[('item', 'http://www.wikidata.org/entity/Q929498'), ('oname', 'Quintus Caecilius Metellus')]
[('item', 'http://www.wikidata.org/entity/Q519186'), ('oname', 'Quintus Hortensius')]
[('item', 'http://www.wikidata.org/entity/Q637596'), ('oname', 'Quintus Servilius Priscus Structus Fidenas')]
[('item', 'http://www.wikidata.org/entity/Q1223522'), ('oname', 'Caius Poetelius Libo Visolus')]
[('item', 'http://www.wikidata.org/entity/Q682087'), ('oname', 'Gaiu

## Task 3
Identify the BGP for Roman Empire and Roman Republic

In [47]:
# write your queries
# Q1747689 --> 'Ancient Rome'
# Q2277 --> 'Roman Empire'
# Q17167 --> 'Roman Republic'
queryString = """
SELECT DISTINCT ?p2name ?REname ?p3name ?RRname
WHERE {
   # bind something
   wd:Q1747689 ?p ?o .
   
   #Roman Empire part  
   {?o ?p1 wd:Q2277 .
   wd:Q2277 ?p2 ?RE .
   ?p2 sc:name ?p2name .
   ?RE sc:name ?REname .}
   
   #wd:Q2277 
   # get the label
   #?s sc:name ?sname .
  
UNION 
        #Roman Republic part
        {?o ?p1 wd:Q17167 .
        wd:Q17167 ?p3 ?RR .
        ?p3 sc:name ?p3name .
        ?RR sc:name ?RRname .}   
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p2name', 'basic form of government'), ('REname', 'autocracy')]
[('p2name', 'basic form of government'), ('REname', 'monarchy')]
[('p2name', 'described by source'), ('REname', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p2name', 'described by source'), ('REname', 'The Nuttall Encyclopædia')]
[('p2name', 'participant in'), ('REname', 'Battles of Bedriacum')]
[('p2name', 'replaces'), ('REname', 'Roman Republic')]
[('p2name', 'replaced by'), ('REname', 'Byzantine Empire')]
[('p2name', 'replaced by'), ('REname', 'Western Roman Empire (395-476 AD)')]
[('p2name', 'religion'), ('REname', 'ancient Roman religion')]
[('p2name', 'religion'), ('REname', 'state church of the Roman Empire')]
[('p2name', 'contains administrative territorial entity'), ('REname', 'Egypt')]
[('p2name', 'country'), ('REname', 'Ancient Rome')]
[('p2name', 'category of associated people'), ('REname', 'Category:People of the Roman Empire')]
[('p2name', 'different from'), ('REname', 'Latin Empire')]
[

Final query for this task

In [41]:
# write your final query
# Q1747689 --> 'Ancient Rome'
# Q2277 --> 'Roman Empire'
# Q17167 --> 'Roman Republic'
queryString = """
SELECT DISTINCT ?p2 ?p2name ?RE ?REname ?p3 ?p3name ?RR ?RRname
WHERE {
   # bind something
   wd:Q1747689 ?p ?o .
   
   #Roman Empire part  
   {?o ?p1 wd:Q2277 .
   wd:Q2277 ?p2 ?RE .
   ?p2 sc:name ?p2name .
   ?RE sc:name ?REname .}
   
   #wd:Q2277 
   # get the label
   #?s sc:name ?sname .
  
UNION 
        #Roman Republic part
        {?o ?p1 wd:Q17167 .
        wd:Q17167 ?p3 ?RR .
        ?p3 sc:name ?p3name .
        ?RR sc:name ?RRname .}   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p2', 'http://www.wikidata.org/prop/direct/P122'), ('p2name', 'basic form of government'), ('RE', 'http://www.wikidata.org/entity/Q173424'), ('REname', 'autocracy')]
[('p2', 'http://www.wikidata.org/prop/direct/P122'), ('p2name', 'basic form of government'), ('RE', 'http://www.wikidata.org/entity/Q7269'), ('REname', 'monarchy')]
[('p2', 'http://www.wikidata.org/prop/direct/P1343'), ('p2name', 'described by source'), ('RE', 'http://www.wikidata.org/entity/Q19180675'), ('REname', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p2', 'http://www.wikidata.org/prop/direct/P1343'), ('p2name', 'described by source'), ('RE', 'http://www.wikidata.org/entity/Q3181656'), ('REname', 'The Nuttall Encyclopædia')]
[('p2', 'http://www.wikidata.org/prop/direct/P1344'), ('p2name', 'participant in'), ('RE', 'http://www.wikidata.org/entity/Q63146684'), ('REname', 'Battles of Bedriacum')]
[('p2', 'http://www.wikidata.org/prop/direct/P1365'), ('p2name', 'replaces'), ('RE', 'http://www.wiki

## Task 4
Return the IRI and the name of all Roman emperors killed by homicide (see "manner of death") and, for each killed emperor, return also the IRI and name of the killer, if present in Wikidata.

In [28]:
# write your queries
# Q842606 --> 'Roman emperor' 
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # bind something
   wd:Q842606 wdt:P2354 ?o  .
   # get the label
   #?s sc:name ?sname .
   #?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q125740'), ('oname', 'list of Roman emperors')]
1


In [32]:
# write your queries
# Q842606 --> 'Roman emperor' 
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # bind something
    ?o ?s wd:Q125740.
   # get the label
   #?s sc:name ?sname .
   #?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q842606'), ('oname', 'Roman emperor')]
[('o', 'http://www.wikidata.org/entity/Q7361954'), ('oname', 'Roman Emperor')]
[('o', 'http://www.wikidata.org/entity/Q6992597'), ('oname', 'Category:Roman emperors')]
[('o', 'http://www.wikidata.org/entity/Q7361952'), ('oname', 'Roman Emperor')]
[('o', 'http://www.wikidata.org/entity/Q7361953'), ('oname', 'Roman Emperor')]
5


In [33]:
# write your queries
# Q842606 --> 'Roman emperor' 
queryString = """
SELECT DISTINCT ?item ?oname
WHERE {
   # bind something
   ?item wdt:P31 wd:Q5; 
    wdt:P39 wd:Q842606.
   # get the label
   #?s sc:name ?sname .
   #?p sc:name ?pname.
   ?item sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('item', 'http://www.wikidata.org/entity/Q46780'), ('oname', 'Aurelian')]
[('item', 'http://www.wikidata.org/entity/Q159798'), ('oname', 'Honorius')]
[('item', 'http://www.wikidata.org/entity/Q437604'), ('oname', 'Julius Saturninus')]
[('item', 'http://www.wikidata.org/entity/Q211396'), ('oname', 'Magnus Maximus')]
[('item', 'http://www.wikidata.org/entity/Q550622'), ('oname', 'Maximus of Hispania')]
[('item', 'http://www.wikidata.org/entity/Q1777'), ('oname', 'Maximinus Thrax')]
[('item', 'http://www.wikidata.org/entity/Q518890'), ('oname', 'Gaius Julius Verus Maximus')]
[('item', 'http://www.wikidata.org/entity/Q172471'), ('oname', 'Valens')]
[('item', 'http://www.wikidata.org/entity/Q314828'), ('oname', 'Vetranio')]
[('item', 'http://www.wikidata.org/entity/Q187489'), ('oname', 'Silvanus')]
[('item', 'http://www.wikidata.org/entity/Q1427'), ('oname', 'Hadrian')]
[('item', 'http://www.wikidata.org/entity/Q172168'), ('oname', 'Galerius')]
[('item', 'http://www.wikidata.org/en

In [35]:
# write your queries
# Q842606 --> 'Roman emperor'
# P1196 --> 'manner of death'
# Q149086 --> 'homicide'
queryString = """
SELECT DISTINCT ?item ?oname
WHERE {
   # bind something
   ?item wdt:P31 wd:Q5; 
    wdt:P39 wd:Q842606;
    wdt:P1196 wd:Q149086.
    
   # get the label
   #?s sc:name ?sname .
   #?p sc:name ?pname.
   ?item sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('item', 'http://www.wikidata.org/entity/Q46780'), ('oname', 'Aurelian')]
[('item', 'http://www.wikidata.org/entity/Q437604'), ('oname', 'Julius Saturninus')]
[('item', 'http://www.wikidata.org/entity/Q1777'), ('oname', 'Maximinus Thrax')]
[('item', 'http://www.wikidata.org/entity/Q518890'), ('oname', 'Gaius Julius Verus Maximus')]
[('item', 'http://www.wikidata.org/entity/Q1762'), ('oname', 'Elagabalus')]
[('item', 'http://www.wikidata.org/entity/Q1411'), ('oname', 'Claudius')]
[('item', 'http://www.wikidata.org/entity/Q1423'), ('oname', 'Domitian')]
[('item', 'http://www.wikidata.org/entity/Q185538'), ('oname', 'Constans')]
[('item', 'http://www.wikidata.org/entity/Q46734'), ('oname', 'Constantine II')]
[('item', 'http://www.wikidata.org/entity/Q103860'), ('oname', 'Julius Nepos')]
[('item', 'http://www.wikidata.org/entity/Q1830'), ('oname', 'Decius')]
[('item', 'http://www.wikidata.org/entity/Q104475'), ('oname', 'Gallienus')]
[('item', 'http://www.wikidata.org/entity/Q1891

Final query for this task

I checked the datail of roman emperor and i get all data execpt killer name so killer name is not available in the wikidata.

In [36]:
# write your final query
# Q842606 --> 'Roman emperor'
# P1196 --> 'manner of death'
# Q149086 --> 'homicide'
queryString = """
SELECT DISTINCT ?item ?oname
WHERE {
   # bind something
   #Get the data of julius Caesar from the given data
   ?item wdt:P31 wd:Q5; 
    wdt:P39 wd:Q842606;
    wdt:P1196 wd:Q149086.
    
   # get the label
   #?s sc:name ?sname .
   #?p sc:name ?pname.
   ?item sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('item', 'http://www.wikidata.org/entity/Q46780'), ('oname', 'Aurelian')]
[('item', 'http://www.wikidata.org/entity/Q437604'), ('oname', 'Julius Saturninus')]
[('item', 'http://www.wikidata.org/entity/Q1777'), ('oname', 'Maximinus Thrax')]
[('item', 'http://www.wikidata.org/entity/Q518890'), ('oname', 'Gaius Julius Verus Maximus')]
[('item', 'http://www.wikidata.org/entity/Q1762'), ('oname', 'Elagabalus')]
[('item', 'http://www.wikidata.org/entity/Q1411'), ('oname', 'Claudius')]
[('item', 'http://www.wikidata.org/entity/Q1423'), ('oname', 'Domitian')]
[('item', 'http://www.wikidata.org/entity/Q185538'), ('oname', 'Constans')]
[('item', 'http://www.wikidata.org/entity/Q46734'), ('oname', 'Constantine II')]
[('item', 'http://www.wikidata.org/entity/Q103860'), ('oname', 'Julius Nepos')]
[('item', 'http://www.wikidata.org/entity/Q1830'), ('oname', 'Decius')]
[('item', 'http://www.wikidata.org/entity/Q104475'), ('oname', 'Gallienus')]
[('item', 'http://www.wikidata.org/entity/Q1891

## Task 5
Return the Roman emperors who killed another Roman emperor (the result set must a list of 4-tuples with killer IRI and label, and killed IRI and label).

In [42]:
# write your queries
# Q2277 --> 'Roman Empire'
# Q1048 --> 'Julius Caesar'
queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE {
   # bind something
   #Get the data of julius Caesar from the given data
    wd:Q1048 ?s ?p .
   # get the label
   ?s sc:name ?sname .
   ?p sc:name ?pname.
   #?o sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P39'), ('sname', 'position held'), ('p', 'http://www.wikidata.org/entity/Q5413489'), ('pname', 'curule aedile')]
[('s', 'http://www.wikidata.org/prop/direct/P3701'), ('sname', 'incarnation of'), ('p', 'http://www.wikidata.org/entity/Q58373'), ('pname', 'Amun')]
[('s', 'http://www.wikidata.org/prop/direct/P800'), ('sname', 'notable work'), ('p', 'http://www.wikidata.org/entity/Q544948'), ('pname', 'Commentarii de Bello Civili')]
[('s', 'http://www.wikidata.org/prop/direct/P3716'), ('sname', 'social classification'), ('p', 'http://www.wikidata.org/entity/Q16744001'), ('pname', 'noble')]
[('s', 'http://www.wikidata.org/prop/direct/P2358'), ('sname', 'Roman praenomen'), ('p', 'http://www.wikidata.org/entity/Q5517401'), ('pname', 'Gaius')]
[('s', 'http://www.wikidata.org/prop/direct/P39'), ('sname', 'position held'), ('p', 'http://www.wikidata.org/entity/Q172907'), ('pname', 'praetor')]
[('s', 'http://www.wikidata.org/prop/direct/P39'), ('

In [46]:
# write your queries
# Q842606 --> 'Roman emperor' 
# P157 -->'killed by' 
queryString = """
SELECT DISTINCT ?item ?emperorname ?ki ?killername
WHERE {
   # bind something  
    ?item wdt:P31 wd:Q5; 
     wdt:P39 wd:Q842606;
     wdt:P157 ?ki.
     
     ?ki wdt:P39 wd:Q842606.
   
   # get the label
   ?item sc:name ?emperorname .
   ?ki sc:name ?killername.
   #?o sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('item', 'http://www.wikidata.org/entity/Q184549'), ('emperorname', 'Licinius'), ('ki', 'http://www.wikidata.org/entity/Q8413'), ('killername', 'Constantine the Great')]
[('item', 'http://www.wikidata.org/entity/Q183089'), ('emperorname', 'Geta'), ('ki', 'http://www.wikidata.org/entity/Q1446'), ('killername', 'Caracalla')]
2


Final query for this task

In [47]:
# write your final query
queryString = """
SELECT DISTINCT ?item ?emperorname ?ki ?killername
WHERE {
   # bind something
   #Get the data of julius Caesar from the given data
    
    ?item wdt:P31 wd:Q5; 
     wdt:P39 wd:Q842606;
     wdt:P157 ?ki.
     
     ?ki wdt:P39 wd:Q842606.
   
   # get the label
   ?item sc:name ?emperorname .
   ?ki sc:name ?killername.
   #?o sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('item', 'http://www.wikidata.org/entity/Q184549'), ('emperorname', 'Licinius'), ('ki', 'http://www.wikidata.org/entity/Q8413'), ('killername', 'Constantine the Great')]
[('item', 'http://www.wikidata.org/entity/Q183089'), ('emperorname', 'Geta'), ('ki', 'http://www.wikidata.org/entity/Q1446'), ('killername', 'Caracalla')]
2


## Task 6
Find the BGP for war

In [55]:
# write your queries
# Q202161 --> 'Gallic War' 
#P31 --> Instance
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # bind something
   wd:Q202161 wdt:P31 ?p.
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   #?o sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q198'), ('pname', 'war')]
1


In [56]:
# write your queries 
# Q198 --> 'war'
# Q202161 --> 'Gallic War' 
# P31 --> Instance
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something 
   wd:Q198 ?p ?o.
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q350604'), ('oname', 'armed conflict')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pname', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q454'), ('oname', 'peace')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8394144'), ('oname', 'Category:War')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('oname', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1190554'), ('oname', 'occurrence')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q178561'), ('oname', 'battle')]
[('p', 'http://www.wikidata.or

Final query for this task

In [57]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   #Get the data of julius Caesar from the given data
   wd:Q198 ?p ?o.
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q350604'), ('oname', 'armed conflict')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pname', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q454'), ('oname', 'peace')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8394144'), ('oname', 'Category:War')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('oname', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1190554'), ('oname', 'occurrence')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q178561'), ('oname', 'battle')]
[('p', 'http://www.wikidata.or

## Task 7
Find the wars (note that "war" and "battle" are not the same thing) participated by the Roman Empire (return IRI and name of the war).

In [59]:
# write your queries
# Q2277 --> 'Roman Empire' from the above query
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q2277 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pname', 'contains administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q202311'), ('oname', 'Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P38'), ('pname', 'currency'), ('o', 'http://www.wikidata.org/entity/Q952064'), ('oname', 'Roman currency')]
[('p', 'http://www.wikidata.org/prop/direct/P38'), ('pname', 'currency'), ('o', 'http://www.wikidata.org/entity/Q208041'), ('oname', 'sestertius')]
[('p', 'http://www.wikidata.org/prop/direct/P237'), ('pname', 'coat of arms'), ('o', 'http://www.wikidata.org/entity/Q163323'), ('oname', 'Roman legion')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q1456601'), ('oname', 'Category:Roman Empire')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11514315'), ('oname', 'historical period')]
[('p', 'http://www.wikida

In [65]:
# write your queries
# Q2277 --> 'Roman Empire' from the above query
# P1344 --> 'participant in'
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # bind something
   wd:Q2277 wdt:P1344 ?o .
   # get the label
   #?s sc:name ?sname .
   #?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q63146684'), ('oname', 'Battles of Bedriacum')]
1


In [72]:
# write your queries
# Q2277 --> 'Roman Empire' from the above query
# P1344 --> 'participant in'
# P4224 --> 'category contains'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   ?o wdt:P4224 wd:Q198 .
   # get the label
   #?s sc:name ?sname .
   #?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q6324312'), ('oname', 'Category:Wars involving Turkey')]
[('o', 'http://www.wikidata.org/entity/Q6324267'), ('oname', 'Category:Wars involving Greece')]
[('o', 'http://www.wikidata.org/entity/Q6990096'), ('oname', 'Category:Wars involving Pakistan')]
[('o', 'http://www.wikidata.org/entity/Q6480612'), ('oname', 'Category:Wars involving Ethiopia')]
[('o', 'http://www.wikidata.org/entity/Q6990260'), ('oname', 'Category:Wars involving Poland')]
[('o', 'http://www.wikidata.org/entity/Q6989300'), ('oname', 'Category:Wars involving ancient Rome')]
[('o', 'http://www.wikidata.org/entity/Q6802131'), ('oname', 'Category:Wars involving the Soviet Union')]
[('o', 'http://www.wikidata.org/entity/Q6244273'), ('oname', 'Category:Wars involving Israel')]
[('o', 'http://www.wikidata.org/entity/Q8085859'), ('oname', 'Category:Wars between the United States and Native Americans')]
9


In [73]:
# write your queries
# Q2277 --> 'Roman Empire' from the above query
# P1344 --> 'participant in'
# P4224 --> 'category contains'
# Q6989300 --> 'Category:Wars involving ancient Rome'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q6989300 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P4224'), ('pname', 'category contains'), ('o', 'http://www.wikidata.org/entity/Q198'), ('oname', 'war')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4167836'), ('oname', 'Wikimedia category')]
2


In [83]:
# write your queries
# Q2277 --> 'Roman Empire' from the above query
# P1344 --> 'participant in'
# P4224 --> 'category contains'
# Q6989300 --> 'Category:Wars involving ancient Rome'
# Q198 --> 'war' 
#P31 --> Instance
queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # bind something
   ?s wdt:P31 wd:Q198.
   # get the label
   ?s sc:name ?sname .
   #?p sc:name ?pname.
   #?o sc:name ?oname
   Filter(Regex (?sname ,"Roman*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q3417663'), ('sname', 'Roman–Gallic wars')]
[('s', 'http://www.wikidata.org/entity/Q252298'), ('sname', 'Hungarian–Romanian war of 1919')]
[('s', 'http://www.wikidata.org/entity/Q4205368'), ('sname', 'Jewish–Roman wars')]
[('s', 'http://www.wikidata.org/entity/Q486831'), ('sname', 'Roman–Seleucid War')]
[('s', 'http://www.wikidata.org/entity/Q500409'), ('sname', 'Roman–Persian Wars')]
[('s', 'http://www.wikidata.org/entity/Q202578'), ('sname', 'First Jewish-Roman War')]
[('s', 'http://www.wikidata.org/entity/Q1533828'), ('sname', 'Roman–Parthian War of 58–63')]
[('s', 'http://www.wikidata.org/entity/Q3778687'), ('sname', 'Romanian War of Independence')]
[('s', 'http://www.wikidata.org/entity/Q2407716'), ('sname', 'Roman–Sassanid War')]
[('s', 'http://www.wikidata.org/entity/Q1790268'), ('sname', 'Roman–Parthian War of 161–166')]
[('s', 'http://www.wikidata.org/entity/Q84850006'), ('sname', 'Roman–Parthian War of 195–198')]
[('s', 'http://w

In [88]:
# write your queries
# Q2277 --> 'Roman Empire' from the above query
# P1344 --> 'participant in'
# P4224 --> 'category contains'
# Q6989300 --> 'Category:Wars involving ancient Rome'
# Q198 --> 'war' 
#P31 --> Instance
# Q500409 --> 'Roman–Persian Wars' 
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname 
WHERE {
   # bind something
    wd:Q500409 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('o', 'http://www.wikidata.org/entity/Q202311'), ('oname', 'Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('o', 'http://www.wikidata.org/entity/Q12824780'), ('oname', 'Asia Minor')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('o', 'http://www.wikidata.org/entity/Q81483'), ('oname', 'Levant')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('pname', 'participant'), ('o', 'http://www.wikidata.org/entity/Q83891'), ('oname', 'Sasanian Empire')]
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('pname', 'has effect'), ('o', 'http://www.wikidata.org/entity/Q275918'), ('oname', 'Muslim conquests')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q1192043'), ('oname', 'Roman–Parthian Wars')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o

Final query for this task

In [89]:
# Q2277 --> 'Roman Empire' from the above query
# P1344 --> 'participant in'
# P4224 --> 'category contains'
# Q6989300 --> 'Category:Wars involving ancient Rome'
# Q198 --> 'war' 
#P31 --> Instance
# Q500409 --> 'Roman–Persian Wars' 
# P710 --> 'participant'
# Q198 --> 'war'
queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # bind something
   ?s wdt:P31 wd:Q198;
   wdt:P710 wd:Q2277.
   
   # get the label
   ?s sc:name ?sname .
   #?p sc:name ?pname.
   #?o sc:name ?oname
   Filter(Regex (?sname ,"Roman*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)
    


Results
[('s', 'http://www.wikidata.org/entity/Q500409'), ('sname', 'Roman–Persian Wars')]
[('s', 'http://www.wikidata.org/entity/Q202578'), ('sname', 'First Jewish-Roman War')]
[('s', 'http://www.wikidata.org/entity/Q1533828'), ('sname', 'Roman–Parthian War of 58–63')]
[('s', 'http://www.wikidata.org/entity/Q1790268'), ('sname', 'Roman–Parthian War of 161–166')]
[('s', 'http://www.wikidata.org/entity/Q84850006'), ('sname', 'Roman–Parthian War of 195–198')]
5


## Task 8
Consider all the wars participated by the Roman Empire. For each war return the number of related battles and the opponent (a war is composed of one or more battles) (the result set must be a list of 5-tuples war IRI, label, opponent IRI and label, and #battles).

In [91]:
# write your queries
# P527 --> 'has part' 
# Q500409 --> 'Roman–Persian Wars'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname 
WHERE {
   # bind something
   wd:Q500409 wdt:P527 ?o .
   # get the label
   #?s sc:name ?sname .
   #?p sc:name ?pname.
   ?o sc:name ?oname.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)




Results
[('o', 'http://www.wikidata.org/entity/Q1192043'), ('oname', 'Roman–Parthian Wars')]
[('o', 'http://www.wikidata.org/entity/Q858108'), ('oname', 'Byzantine–Sassanian Wars')]
2


In [92]:
# write your queries
# P527 --> 'has part' 
# Q500409 --> 'Roman–Persian Wars'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname 
WHERE {
   # bind something
   wd:Q500409 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname.
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)




Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('o', 'http://www.wikidata.org/entity/Q202311'), ('oname', 'Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('o', 'http://www.wikidata.org/entity/Q12824780'), ('oname', 'Asia Minor')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('o', 'http://www.wikidata.org/entity/Q81483'), ('oname', 'Levant')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('pname', 'participant'), ('o', 'http://www.wikidata.org/entity/Q83891'), ('oname', 'Sasanian Empire')]
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('pname', 'has effect'), ('o', 'http://www.wikidata.org/entity/Q275918'), ('oname', 'Muslim conquests')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q1192043'), ('oname', 'Roman–Parthian Wars')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o

Final query for this task

In [94]:
# write your final query

There is no data avaiable of any battle related to any war